imports

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

%cd '/content/drive/My Drive/Colab Notebooks'

!pip install import-ipynb

import import_ipynb

from bce_loss_functions import *

from cce_loss_functions import *

from custom_metrics import *

from hmda_data_loader import *

import pandas as pd

import numpy as np

import random as rd

import tensorflow as tf

from tensorflow.python.framework import constant_op

from tensorflow import keras

from keras import layers

from scipy.stats import chi2_contingency

constants and variables

In [ ]:
DIRECTORIES = {
    'MODELS':'/content/drive/MyDrive/Colab Notebooks/models/hmda/',
    'OUTPUT':'/content/drive/MyDrive/Colab Notebooks/ouput/hmda/'
}

load data

In [ ]:
category = 'race'

print('\nloading training data\n')

train_X,train_Y,train_ = get_data(dataset_type='train',category=category,normalization='std')

train_data = tf.convert_to_tensor(train_X,dtype='float32')
train_targets = tf.convert_to_tensor(train_Y,dtype='float32')

train_Xc = train_[0]

train_data_c = tf.convert_to_tensor(train_Xc,dtype='float32')

print('train accept:\t\t',int(np.sum(train_[1])))
print('train denied:\t\t',len(train_[1]) - int(np.sum(train_[1])))
print('train accept rate:\t',np.sum(train_[1])/len(train_[1]))

print(train_data.shape)
print(train_targets.shape)
print(train_data_c.shape)

create model function

In [ ]:
def create_model(name='model',train_data=None,train_targets=None,activation='sigmoid',loss='binary crossentropy',learning_rate=0.01,num_epochs=11,batch_size=128,steps_per_epoch=None,callbacks=None,verbose=1):

        model_path = DIRECTORIES['MODELS'] + name

        model = keras.Sequential([layers.Dense(1, activation=activation)])

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss=loss, metrics=[CBA(),CTP(),CTN(),CFP(),CFN()])

        print('fit model')

        history = model.fit(train_data, train_targets, validation_split=0.2, shuffle=True, epochs=num_epochs, batch_size=batch_size,  steps_per_epoch=steps_per_epoch, verbose=verbose, callbacks=callbacks)

        print('acc:\t',history.history['custom_binary_accuracy'][-1])
        print('loss:\t',history.history['loss'][-1])

        print('tp history:\t',history.history['custom_true_positives'][-1])
        print('tn history:\t',history.history['custom_true_negatives'][-1])
        print('fp history:\t',history.history['custom_false_positives'][-1])
        print('fn history:\t',history.history['custom_false_negatives'][-1])

        model.save(model_path)

        return model

In [ ]:
callbacks = None

num_epochs = 11

batch_size = 128

steps_per_epoch = ((len(train_data) * 0.8) // batch_size) + 1

activation = 'sigmoid'

learning_rate = 0.01

verbose = 1

models_b = {}

models_c = {}

print('\n\nbatch size:\t\t{}\nmax number of epochs:\t{}\nsteps per epoch:\t{}'.format(batch_size,num_epochs,steps_per_epoch))

for alpha in [0,0.25,0.5,0.75]:

    print('\n\nalpha value:\t\t',alpha)

    losses = {
        'v1:MGR_Equity_BCE':MaxGroupRiskBCEv1(alpha=alpha),
        'v2:MGR_Equality_BCE':MaxGroupRiskBCEv2(alpha=alpha),
    }

    for ver,loss in losses.items():

        model_name = ver + '_' + str(alpha)

        print('\n\ncreating {} alpha = {} loss function base model'.format(ver,alpha))

        models_b[model_name] = create_model(model_name + '_hmda_ca_22_{}_b.keras'.format(category),train_data,train_targets,activation,loss,learning_rate,num_epochs,batch_size,steps_per_epoch,callbacks,verbose)

        print('\n\ncreating {} alpha = {} loss function category model'.format(ver,alpha))

        models_c[model_name] = create_model(model_name + '_hmda_ca_22_{}_c.keras'.format(category),train_data_c,train_targets,activation,loss,learning_rate,num_epochs,batch_size,steps_per_epoch,callbacks,verbose)

